In [ ]:
import pandas as pd

In [ ]:
segdup = pd.read_csv("../data/segdup_regions.csv")[["chrom", "chromStart", "chromEnd", "fracMatch", "fracMatchIndel", "otherChrom", "otherStart", "otherEnd"]]

In [ ]:
segdup

## convert to bed

In [ ]:
segdup = segdup[(segdup.fracMatch > 0.98) & (~segdup.chrom.str.contains("_"))]

In [ ]:
segdup["chrom"] = segdup["chrom"].replace({"chrX": "chr23", "chrY": "chr24", "chrM": "chr25"})
segdup["chrom"] = segdup["chrom"].str.replace("chr", "").astype("int")
segdup = segdup.sort_values(by=["chrom", "chromStart", "chromEnd"])
segdup["chrom"] = "chr" + segdup["chrom"].astype(str)
segdup["chrom"] = segdup["chrom"].replace({"chr23": "chrX", "chr24": "chrY", "chr25": "chrM"})

In [ ]:
segdup

In [ ]:
segdup.otherChrom.value_counts()

In [ ]:
segdup[["chrom", "chromStart", "chromEnd"]].to_csv("../data/segDup_majorAllele_withAltContigs_98pcFracMatch.bed", sep="\t", header=False, index=False)

### a lot of segments in this bed file overlap with each other. had to run bedtools merge to get the union of those segments

## see how many genes in the CN matrix are going to be masked

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

OmicsCNGene = tc.get(name='internal-23q2-1e49', version=97, file='OmicsCNGene')


In [ ]:
from mgenepy.utils import helper as h

mybiomart = h.generateGeneNames(attributes=["start_position", "end_position", "chromosome_name"],)
mybiomart = mybiomart.rename(
        columns={
            "start_position": "start",
            "end_position": "end",
            "chromosome_name": "Chromosome"
        }
    )

In [ ]:
mybiomart["Chromosome"] = mybiomart["Chromosome"].replace({"X": "23", "Y": "24", "MT": "25"})
mybiomart = mybiomart[mybiomart.Chromosome.isin(set(map(str, range(1, 26))))]
mybiomart["Chromosome"] = mybiomart["Chromosome"].astype(int)

In [ ]:
mybiomart = mybiomart.sort_values(by=["Chromosome", "start", "end"])
mybiomart["Chromosome"] = mybiomart["Chromosome"].replace({23: "X", 24: "Y", 25: "MT"})
mybiomart = mybiomart.drop_duplicates("hgnc_symbol", keep="first")
mybiomart["gene_name"] = [
        i["hgnc_symbol"] + " (" + str(i["entrezgene_id"]).split(".")[0] + ")"
        for _, i in mybiomart.iterrows()
    ]
mybiomart["Chromosome"] = "chr" + mybiomart["Chromosome"].astype(str)


In [ ]:
mybiomart

In [ ]:
biomart_exon_raw = h.generateGeneNames(attributes=["chromosome_name", "exon_chrom_start", "exon_chrom_end", "ensembl_gene_id"], default_attr=[])

In [ ]:
len(biomart_exon_raw.ensembl_gene_id.unique())

In [ ]:
biomart_exon = biomart_exon_raw.merge(mybiomart[["ensembl_gene_id", "gene_name"]], on=["ensembl_gene_id"], how="left")
biomart_exon = biomart_exon[~biomart_exon.gene_name.isin(OmicsCNGene)]

In [ ]:
biomart_exon["chromosome_name"] = biomart_exon["chromosome_name"].replace({"X": "23", "Y": "24", "MT": "25"})
biomart_exon = biomart_exon[biomart_exon.chromosome_name.isin(set(map(str, range(1, 26))))]
biomart_exon["chromosome_name"] = biomart_exon["chromosome_name"].astype(int)
biomart_exon = biomart_exon.sort_values(by=["chromosome_name", "exon_chrom_start", "exon_chrom_end"])
biomart_exon["chromosome_name"] = biomart_exon["chromosome_name"].replace({23: "X", 24: "Y", 25: "MT"})
biomart_exon["chromosome_name"] = "chr" + biomart_exon["chromosome_name"].astype(str)


In [ ]:
biomart_exon[['chromosome_name', 'exon_chrom_start', 'exon_chrom_end', 'gene_name']].to_csv("biomart_exons.bed", sep="\t", header=False, index=False)

In [ ]:
biomart_exon

In [ ]:
exons = pybedtools.BedTool("biomart_exons.bed")
exons.merge().saveas('biomart_exons_merged.bed')

In [ ]:
mybiomart.merge(biomart_exon, on=["ensembl_gene_id"], how="right")

In [ ]:
def exonUnion(df):
    l = list(zip(df.exon_chrom_start, df.exon_chrom_end))
    
    sorted_data = sorted(l)
    b = []
    for begin,end in sorted_data:
        if b and b[-1][1] >= begin - 1:
            b[-1][1] = max(b[-1][1], end)
        else:
            b.append([begin, end])
    return b

In [ ]:
df = pd.DataFrame(biomart_exon.groupby(["gene_name"]).apply(exonUnion), columns=["exons"]).reset_index()

In [ ]:
df

In [ ]:
df_ = df.explode('exons').reset_index(drop=True)

In [ ]:
df_ = pd.concat(
    [df_[['gene_name']], pd.DataFrame(df_['exons'].tolist()).add_prefix('col')], axis=1
)
df_.merge(mybiomart[['gene_name', 'Chromosome']], on=["gene_name"], how="left")

In [ ]:
biomart_exon_union

In [ ]:
#biomart_exon_raw = h.generateGeneNames(attributes=["chromosome_name", "exon_chrom_start", "exon_chrom_end", "ensembl_gene_id"], default_attr=[])
biomart_exon = biomart_exon_raw.merge(mybiomart[["ensembl_gene_id", "gene_name"]], on=["ensembl_gene_id"], how="left")
biomart_exon = biomart_exon[~biomart_exon.gene_name.isin(OmicsCNGene)]

biomart_exon_union = pd.DataFrame(biomart_exon.groupby(["gene_name"]).apply(exonUnion), columns=["exons"]).reset_index().explode('exons').reset_index(drop=True)
biomart_exon_union = pd.concat([biomart_exon_union[['gene_name']], pd.DataFrame(biomart_exon_union['exons'].tolist()).add_prefix('col')], axis=1)
biomart_exon_union = biomart_exon_union.merge(biomart_exon[['gene_name', 'chromosome_name']], on=["gene_name"], how="left")
biomart_exon_union["chromosome_name"] = biomart_exon_union["chromosome_name"].replace({"X": "23", "Y": "24", "MT": "25"})
biomart_exon_union = biomart_exon_union[biomart_exon_union.chromosome_name.isin(set(map(str, range(1, 26))))]
biomart_exon_union["chromosome_name"] = biomart_exon_union["chromosome_name"].astype(int)
biomart_exon_union = biomart_exon_union.sort_values(by=["chromosome_name", "col0", "col1"])
biomart_exon_union["chromosome_name"] = biomart_exon_union["chromosome_name"].replace({23: "X", 24: "Y", 25: "MT"})
biomart_exon_union["chromosome_name"] = "chr" + biomart_exon_union["chromosome_name"].astype(str)

In [ ]:
biomart_exon_union

In [ ]:
cnmatrix, masked_genes_segdup, masked_genes_rm = maskSegDup(OmicsCNGene, "CN", mybiomart, thresh=0.5, repeat_bed="/home/xiaomeng/bin/depmap_omics/data/repeatMasker_max10_noAlt_merged.bed")

In [ ]:
cnmatrix

In [ ]:
exon_union_mat = biomart_exon.groupby('ensembl_gene_id').apply(exonUnion)

In [ ]:
import pandas as pd

exon_union_mat = pd.DataFrame(exon_union_mat.reset_index(), columns=["ensembl_gene_id", "exons"])

In [ ]:
exon_union_mat

In [ ]:
mybiomart.merge(exon_union_mat, on=['ensembl_gene_id'], how="left")

In [ ]:
ensmbl.attribute_pages['sequences']

In [ ]:
mybiomart[mybiomart.gene_name.isin(OmicsCNGene)][["Chromosome", "start", "end", "gene_name"]].to_csv("../data/biomart_cngenes.bed", sep="\t", header=False, index=False)

In [ ]:
gene_dict = mybiomart[["Chromosome", "start", "end", "gene_name"]].set_index('gene_name').T.to_dict('list')

In [ ]:
gene_dict

In [ ]:
overlap_df = pd.read_csv("/home/xiaomeng/bin/depmap_omics/data/overlap_withalt.txt", sep="\t", names=["chrom", "start", "end", "gene_name"])

In [ ]:
overlap_df

# a proper function for it

In [ ]:
import pybedtools

def maskSegDup(cnmatrix, matname, mybiomart, save_output="", segdup_bed="../data/segDup_majorAllele_withAltContigs_98pcFracMatch_merged.bed", repeat_bed="../data/repeatMasker_max100_noAlt_merged.bed", thresh=0.5):
    # sort and format biomart
#     mybiomart["Chromosome"] = mybiomart["Chromosome"].replace({"X": "23", "Y": "24", "MT": "25"})
#     mybiomart = mybiomart[mybiomart.Chromosome.isin(set(map(str, range(1, 26))))]
#     mybiomart["Chromosome"] = mybiomart["Chromosome"].astype(int)
#     mybiomart = mybiomart.sort_values(by=["Chromosome", "start", "end"])
#     mybiomart["Chromosome"] = mybiomart["Chromosome"].replace({23: "X", 24: "Y", 25: "MT"})
#     mybiomart = mybiomart.drop_duplicates("hgnc_symbol", keep="first")
#     mybiomart["Chromosome"] = "chr" + mybiomart["Chromosome"].astype(str)
    
    mybiomart = mybiomart[mybiomart.gene_name.isin(cnmatrix)]
    mybiomart[["Chromosome", "start", "end", "gene_name"]].to_csv(save_output + "biomart_cngenes.bed", sep="\t", header=False, index=False)
    cngenes = pybedtools.BedTool(save_output + "biomart_cngenes.bed")
    cngenes.intersect(segdup_bed).saveas(save_output + 'mask_overlap_segdup.bed')
    
    overlap_segdup = pd.read_csv(save_output + 'mask_overlap_segdup.bed', sep="\t", names=["chrom", "start", "end", "gene_name"])
    
    # download and reformat exon info
    biomart_exon_raw = h.generateGeneNames(attributes=["chromosome_name", "exon_chrom_start", "exon_chrom_end", "ensembl_gene_id"], default_attr=[])
    biomart_exon = biomart_exon_raw.merge(mybiomart[["ensembl_gene_id", "gene_name"]], on=["ensembl_gene_id"], how="left")
    biomart_exon = biomart_exon[biomart_exon.gene_name.isin(cnmatrix)]

    biomart_exon_union = pd.DataFrame(biomart_exon.groupby(["gene_name"]).apply(exonUnion), columns=["exons"]).reset_index().explode('exons').reset_index(drop=True)
    biomart_exon_union = pd.concat([biomart_exon_union[['gene_name']], pd.DataFrame(biomart_exon_union['exons'].tolist()).add_prefix('col')], axis=1)
    biomart_exon_union = biomart_exon_union.merge(biomart_exon[['gene_name', 'chromosome_name']], on=["gene_name"], how="left")
    biomart_exon_union["chromosome_name"] = biomart_exon_union["chromosome_name"].replace({"X": "23", "Y": "24", "MT": "25"})
    biomart_exon_union = biomart_exon_union[biomart_exon_union.chromosome_name.isin(set(map(str, range(1, 26))))]
    biomart_exon_union["chromosome_name"] = biomart_exon_union["chromosome_name"].astype(int)
    biomart_exon_union = biomart_exon_union.sort_values(by=["chromosome_name", "col0", "col1"])
    biomart_exon_union["chromosome_name"] = biomart_exon_union["chromosome_name"].replace({23: "X", 24: "Y", 25: "MT"})
    biomart_exon_union["chromosome_name"] = "chr" + biomart_exon_union["chromosome_name"].astype(str)

    biomart_exon_union[['chromosome_name', 'col0', 'col1', 'gene_name']].to_csv("biomart_exons.bed", sep="\t", header=False, index=False)
    exons_merged = pybedtools.BedTool("biomart_exons.bed")
    exons_merged.intersect(repeat_bed).saveas(save_output + 'mask_overlap_rm.bed')
    overlap_rm = pd.read_csv(save_output + 'mask_overlap_rm.bed', sep="\t", names=["chrom", "start", "end", "gene_name"])

    gene_dict = mybiomart[["Chromosome", "start", "end", "gene_name"]].set_index('gene_name').T.to_dict('list')
    
    #segdup
    masked_genes_segdup = []
    for g in overlap_segdup.gene_name.unique().tolist():
        _, start, end = gene_dict[g]
        gene_length = end - start
        overlap_length = 0
        overlap_segments = overlap_segdup[overlap_segdup.gene_name == g]
        for i, v in overlap_segments.iterrows():
            overlap_length += v['end'] - v['start']
        if overlap_length / gene_length > thresh:
            masked_genes_segdup.append(g)
    print("masking " + str(len(masked_genes_segdup)) + " genes from CN matrix due to segmental duplication")
    
    #repeat masker
    masked_genes_rm = []
    for g in overlap_rm.gene_name.unique().tolist():
        all_overlaps = overlap_rm[overlap_rm.gene_name == g]
        exons = biomart_exon_union[biomart_exon_union.gene_name == g]
        exon_length = exons["col1"].sum() - exons["col0"].sum()
        overlap_length = all_overlaps["end"].sum() - all_overlaps["start"].sum()
        if overlap_length / exon_length > thresh:
            masked_genes_rm.append(g)
    print("masking " + str(len(masked_genes_rm)) + " genes from CN matrix due to repeat masker, ")
    print(str(len(set(masked_genes_rm) - set(masked_genes_segdup))) + " of which were not masked by segdup")
    
    cnmatrix = cnmatrix.drop(columns=masked_genes_segdup + masked_genes_rm)
    
    return cnmatrix, masked_genes_segdup, masked_genes_rm

In [ ]:
genes = pybedtools.BedTool("../data/biomart_cngenes.bed")

In [ ]:
genes.intersect("/home/xiaomeng/bin/depmap_omics/data/segDup_majorAllele_98pcFracMatch_merged.bed").saveas('ov.bed')

In [ ]:
cnmatrix, masked_genes = maskSegDup(OmicsCNGene, "CN", mybiomart, thresh=0.01, segdup_bed="/home/xiaomeng/bin/depmap_omics/data/repeatMasker_max100_noAlt_merged.bed")

In [ ]:
masked_genes

## repeatmasker

In [ ]:
repeats = pd.read_csv("/home/xiaomeng/bin/depmap_omics/data/repeatmasker.csv")[["milliDiv", "milliDel", "milliIns", "genoName", "genoStart", "genoEnd"]]

In [ ]:
repeats["max"] = repeats[["milliDiv", "milliDel", "milliIns"]].max(axis=1)

In [ ]:
repeats = repeats[(repeats["max"] <= 10) & (~repeats.genoName.str.contains("_"))]
repeats["genoName"] = repeats["genoName"].replace({"chrX": "chr23", "chrY": "chr24", "chrM": "chr25"})
repeats["genoName"] = repeats["genoName"].str.replace("chr", "").astype("int")
repeats = repeats.sort_values(by=["genoName", "genoStart", "genoEnd"])
repeats["genoName"] = "chr" + repeats["genoName"].astype(str)
repeats["genoName"] = repeats["genoName"].replace({"chr23": "chrX", "chr24": "chrY", "chr25": "chrM"})

In [ ]:
repeats[["genoName", "genoStart", "genoEnd"]].to_csv("../data/repeatMasker_max10_noAlt.bed", sep="\t", header=False, index=False)

In [ ]:
repeats

In [ ]:
repeats[(repeats.genoName == "chr11") & (repeats.genoStart > 55635000) & (repeats.genoStart < 55641309)]

In [ ]:
from biomart import BiomartServer
    
server = BiomartServer("http://nov2020.archive.ensembl.org/biomart")
ensmbl = server.datasets["hsapiens_gene_ensembl"]


In [ ]:
res[(res["Gene start (bp)"] != res["Exon region start (bp)"]) | (res["Gene end (bp)"] != res["Exon region end (bp)"])]